# Script to prepare LD reference data for each ancestry group  

After downloading and processing 1000 genomes project LD panels with `1.download_and_prep_1kg_ld_panels.sh`, this `Julia` script prepares LD reference panels from the 1000 genomes project for each ancestry group as follows  
- keep SNPs with minor allele frequency > 0.05
- keep polymorphic SNPs

After filtering, the sciript keeps the filtered .bed file for each ancestry group and removes any intermediate files. The script does not remove the original vcf files or plink files.  

Julia v.1.8.1 was used for this script.

In [1]:
using Pkg
Pkg.add(["Downloads", "SnpArrays", "CSV", "DataFrames", "Glob"])

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [2]:
using Downloads, SnpArrays, CSV, DataFrames, Glob

In [10]:
isdir("data") || mkdir("data")

"data"

# Preparing filtered .bed files for each ancestry group

## Download sample metadata

In [6]:
# Download sample metadata
beagle = "https://bochet.gcc.biostat.washington.edu/beagle/1000_Genomes_phase3_v5a"
url = joinpath(beagle, "sample_info/integrated_call_samples_v3.20130502.ALL.panel")
meta = last(split(url, "/")) 
isfile("data/$(meta)") || Downloads.download(url, "data/$(meta)")

true

In [7]:
meta = CSV.read("data/integrated_call_samples_v3.20130502.ALL.panel", DataFrame, header=false, skipto = 2)
rename!(meta, [:sample, :pop, :super_pop, :gender])
first(meta, 3)

,sample,pop,super_pop,gender
,String7,String3,String3,String7
1,HG00096,GBR,EUR,male
2,HG00097,GBR,EUR,female
3,HG00099,GBR,EUR,female


## Filter SNPs with maf > 0.05 and mac > 1  
Using the processed 1000 genomes project LD panels (see `1.download_and_prep_1kg_ld_panels.sh` for the process), this section will take care of the following things:
1. subset to an ancestry group
2. filter out SNPs with minor allele frequency < 0.05
3. filter out SNPs with minor allele count < 1

In [11]:
ancestries = ["AFR", "AMR", "EAS", "EUR", "SAS"]
dir = "/u/project/gandalm/shared/refGenomes/1000genomes/chrs"

for chr in 1:23
    chr == 23 ? chr = "X" : chr

    println("################################")
    println("####### processing chr$(chr) #######")
    println("################################")
    
       
    #### the original kgp ####
    basefile = "kgp.clean.chr$(chr)"
    kgp_original = SnpData("$(dir)/$(basefile)")
    size_kgp_original = size(kgp_original) # to be used to compare before/after the filtering
    
    
    #### processing for each ancestry group ####
    for ancestry in ancestries
        if isfile("data/$(basefile).$(ancestry).maf005.mac1.bed")
            println("$(basefile).$(ancestry).maf005.mac1.bed exists; skip")
            continue
        end    
        
        rm_files = []
        kgp = kgp_original
        
        #### Subset to the current ancestry samples ####
        println(" ")
        println("### subsetting to $(ancestry) samples ###")
        println("sample size ref: AFR 661, AMR 347, EAS 504, EUR 503, SAS 489")
        ancestry_data = meta.sample[meta.super_pop .== "$(ancestry)"]
        rowinds = findall(in(ancestry_data), kgp.person_info.iid)

        file = "$(basefile).$(ancestry)"
        SnpArrays.filter(kgp, rowinds, trues(size(kgp)[2]); des = "data/$(file)")
        println(" ")
        
        

        #### get colinds: maf > 0.05 ####
        println("filtering SNPs with maf > 0.05")
        kgp = SnpData("data/$(file)") # the current ancestry samples: $(basefile).$(ancestry)
        println("dim for $(ancestry) file (before maf > 0.05)")
        println(size(kgp))

        file = "$(basefile).$(ancestry).maf005"
        colinds = SnpArrays.filter(kgp.snparray; min_maf = 0.05)[2]
        SnpArrays.filter(kgp, trues(size(kgp)[1]), colinds; des = "data/$(file)")   

        kgp = SnpData("data/$(file)") # current ancestry samples with maf > 0.05: $(basefile).$(ancestry).maf005
        println("dim for $(ancestry) file (after maf > 0.05)")
        println(size(kgp))
        println(" ")


      
        #### get colinds: MAC > 1 ####
        println("filtering SNPs with mac > 1")
        
        # count matrix reference: https://openmendel.github.io/SnpArrays.jl/latest/#Counts
        cnt = counts(kgp.snparray, dims = 1)

        # removes all samples fall into one genotype, or all samples' genotype is missing
        colinds = Int[]
        for j in 1:size(kgp)[2]
            size(kgp)[1] in cnt[:,j] ? nothing : push!(colinds, j)
        end

        snps_num_mac1 = length(colinds)
        println("number of SNPs with mac > 1 (will be kept): $(snps_num_mac1)")
        
        # for each column, check if any of the rows have all samples
        # ref. AFR 661, AMR 347, EAS 504, EUR 503, SAS 489
        # if none of the rows == above ancestry sample size in the column, the column will be kept
        file = "$(basefile).$(ancestry).maf005.mac1"
        SnpArrays.filter(kgp, trues(size(kgp)[1]), colinds; des = "data/$(file)")
        
        
        kgp_filtered = SnpData("data/$(basefile).$(ancestry).maf005.mac1")
        size_kgp_filtered = size(kgp_filtered)
        println("dim for $(ancestry) file (after maf > 0.05, mac > 1)")
        println(size_kgp_filtered)
        println("cf: sample size ref - AFR 661, AMR 347, EAS 504, EUR 503, SAS 489")
        println("cf: dim original - $(size_kgp_original)")
        println(" ")

        println(kgp_filtered)
        println(" ")
        println(" ")


        #### clean up the directory; removing unncessesary ancestry files ####       
        rm_file_list = filter(x -> !occursin(r"mac", x), glob("$(basefile).$(ancestry)*", "data"))
        for file in rm_file_list
            rm_files = push!(rm_files, file)
        end
        rm_files = push!(rm_files, "data/$(basefile).$(ancestry).maf005.mac1.fam")
        rm_files = push!(rm_files, "data/$(basefile).$(ancestry).maf005.mac1.bim")

        println("files to be removed: ")        
        println(length(rm_files))
        println(rm_files)        

        for file in rm_files
            rm(file)
        end
        println(" ")
        println(" ")
    end
    
    println("all ancestries were processed for chr$(chr)")
    println(" ")
    println(" ")
end        

################################
####### processing chr1 #######
################################
 
### subsetting to AFR samples ###
sample size ref: AFR 661, AMR 347, EAS 504, EUR 503, SAS 489
 
filtering SNPs with maf > 0.05
dim for AFR file (before maf > 0.05)
(661, 2459855)
dim for AFR file (after maf > 0.05)
(661, 753532)
 
filtering SNPs with mac > 1
number of SNPs with mac > 1 (will be kept): 753522
dim for AFR file (after maf > 0.05, mac > 1)
(661, 753522)
cf: sample size ref - AFR 661, AMR 347, EAS 504, EUR 503, SAS 489
cf: dim original - (2504, 2459855)
 
SnpData(people: 661, snps: 753522,
snp_info: 
 Row │ chromosome  snpid         genetic_distance  position  allele1  allele2
     │ String      String        Float64           Int64     String   String
─────┼────────────────────────────────────────────────────────────────────────
   1 │ 1           1:10177:A:AC               0.0     10177  AC       A
   2 │ 1           1:10352:T:TA               0.0     10352  TA       T
   